In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import packages

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tqdm import tqdm
import sys

sys.path.insert(1, "drive/MyDrive/vanilla-transformer")

In [3]:
from embd import PositionalEmbedding
from encoder import EncoderLayer_postLN, EncoderLayer_preLN

## Build model

In [15]:
from keras.backend import softmax
BATCH_SIZE = 75
def model():
    inputs = keras.layers.Input((137, 15), batch_size=BATCH_SIZE)
    x = PositionalEmbedding(32)(inputs)
    x = EncoderLayer_postLN(d_model=32, num_heads=128, dff=64)(x)
    x = EncoderLayer_postLN(d_model=32, num_heads=64, dff=32)(x)
    x = keras.layers.GlobalAveragePooling1D(data_format='channels_first')(x)
    x = keras.layers.Dense(5, activation='softmax')(x)
    return keras.Model(inputs, x)

In [16]:
model = model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(75, 137, 15)]           0         
                                                                 
 positional_embedding_1 (Pos  (75, 137, 32)            512       
 itionalEmbedding)                                               
                                                                 
 encoder_layer_post_ln_2 (En  (75, 137, 32)            540928    
 coderLayer_postLN)                                              
                                                                 
 encoder_layer_post_ln_3 (En  (75, 137, 32)            270560    
 coderLayer_postLN)                                              
                                                                 
 global_average_pooling1d_1   (75, 137)                0         
 (GlobalAveragePooling1D)                                  

In [18]:
x = tf.random.normal((BATCH_SIZE, 137, 15))
model(x)

<tf.Tensor: shape=(75, 5), dtype=float32, numpy=
array([[0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.2       , 0.2       , 0.2       , 0.2       

## Load data

In [19]:
data = np.load("mfcc.npz")
X = data["X"]
Y = data["Y"]

x_train = X[0:825]
y_train = Y[0:825]
x_test = X[850:]
y_test = Y[850:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE)

x = x_train[0:BATCH_SIZE]
x_rank = tf.rank(x).numpy()
x_norm_resize_shape = [BATCH_SIZE] + list(tf.ones(tf.rank(x), dtype=tf.int32).numpy())[1:]

## Training

In [20]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [21]:
def perturbation_loss(x, y, from_logits=False):
    return keras.losses.CategoricalCrossentropy(from_logits=from_logits)(x, y)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
learning_rate = CustomSchedule(32)
optimizer = keras.optimizers.experimental.AdamW(learning_rate=learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_loss_metric = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [22]:
eps = 8.     # the perturbation parameter
sig = 1e-5   # initial perturbation StdDev
zeta = 1e-6  # differentiation constant
lamd = 1     # regularization parameter

@tf.function
def training_step(x, label):
    x_p = tf.random.normal(x.shape)
    x_norm = x_p
    for i in range(x_rank-1, 0, -1):
        x_norm = tf.norm(x_norm, ord=2, axis=int(i))
    x_p /= tf.reshape(x_norm, (BATCH_SIZE, 1, 1))
    x_p *= zeta


    with tf.GradientTape() as adversarial_tape:
        adversarial_tape.watch(x_p)
        y_p = model(x + x_p, training=True)
        y = model(x, training=True)
        l = perturbation_loss(y, y_p)
    g = adversarial_tape.gradient(l, x_p)

    g_norm = g
    for i in range(x_rank-1, 0, -1):
        g_norm = tf.norm(g_norm, ord=2, axis=int(i))

    x_p = eps * g / (tf.reshape(g_norm, x_norm_resize_shape) + zeta)

    with tf.GradientTape() as model_tape:
        y_p = model(x + x_p, training=True)
        y = model(x, training=True)
        l = perturbation_loss(y, y_p)    # Recalculate regularization

        logits = model(x, training=True)
        loss = loss_fn(label, logits) + lamd * l / BATCH_SIZE
    grads = model_tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(label, logits)
    return loss, train_acc_metric.result(), l, x_p, g

@tf.function
def pre_train(x, label):
    with tf.GradientTape() as model_tape:
        logits = model(x, training=True)
        loss = loss_fn(label, logits)
    grads = model_tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

In [23]:
train_loss = []
train_metric = []
val_metric = []
val_loss = []
p_loss = []

for i in range(4):
    for step, (x, label) in enumerate(train_dataset):
        pre_train(x, label)

for epoch in tqdm(range(1000)):
    print("\nStart of epoch %d" % (epoch,))
    for step, (x, label) in enumerate(train_dataset):
        loss, train_acc, l, x_p, g = training_step(x, label)

    print(
        "Training loss: %.4f\nTraining metric: %.4f"
        % (float(loss), float(train_acc))
    )
    print("perturbation loss: %.4f" % float(l))

    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
        v_loss = val_loss_metric(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc)))

    train_loss.append(loss)
    train_metric.append(train_acc)
    val_metric.append(val_acc)
    val_loss.append(v_loss)
    p_loss.append(l)

    tl = np.array(train_loss)
    tm = np.array(train_metric)
    vm = np.array(val_metric)
    vl = np.array(val_loss)
    pl = np.array(p_loss)

    np.savez("logs.npz", train_loss=tl, train_acc=tm, val_acc=vm, p_loss=pl, val_loss=vl, x_p=x_p)

  0%|          | 0/1000 [00:00<?, ?it/s]


Start of epoch 0


  0%|          | 1/1000 [00:08<2:21:11,  8.48s/it]

Training loss: 1.6306
Training metric: 0.3370
perturbation loss: 1.6094
Validation acc: 0.3800

Start of epoch 1


  0%|          | 2/1000 [00:12<1:35:02,  5.71s/it]

...

Training loss: 0.0015
Training metric: 0.9632
perturbation loss: 0.0280
Validation acc: 0.9400

Start of epoch 998


100%|█████████▉| 999/1000 [1:02:30<00:03,  3.77s/it]

Training loss: 0.0001
Training metric: 0.9633
perturbation loss: 0.0006
Validation acc: 0.9600

Start of epoch 999


100%|██████████| 1000/1000 [1:02:34<00:00,  3.75s/it]

Training loss: 0.0034
Training metric: 0.9633
perturbation loss: 0.0104
Validation acc: 0.9533
